<h1>Part 03 - Data Management</h1>

<h2>Exercise 01 - Data Quality Test with Great Expectations</h2>


<font size="3">
Goal of this section is to get to know better data management tools, in particular Great Expectation, which allows you to perform data quality assessment & alerting on your day to day projects.

We will continue using the TLC trip record data, deep dive specifically into these datasets to catch data quality issues & encode our own set of rules & triggers.
</font>

## 1 - Getting familiar with the data

### 1-1. Downloading our data

To start with, let's download our data: we will use a larger dataset containing several details about January 2022 "for hire vehicles" trips in NYC (Uber, Lyft...). This dataset has been lightly modified for the purpose of our exercise. Let's download it & save it under our data folder. 

In [ ]:
import gdown
import os

data_folder = "data"

# Check whether the specified path exists or not
isExist = os.path.exists(data_folder)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(data_folder)
    print(f"New directory {data_folder} created!")

gdown.download(
    "https://drive.google.com/uc?id=1xQ8heQzUkKehOUPYvrHIqQ_pDJNCH9tT",
    "data/taxi-trips-2022-01.parquet",
    quiet=False,
)
gdown.download(
    "https://drive.google.com/uc?id=11kOFkDJIXSW2Hu0o2o-PWBhTJi0msYfH",
    "data/taxi-trips-2022-02.parquet",
    quiet=False,
)

### 1-2. Loading our data

Our data is composed of several columns, the most interesting ones being:
- `hvfhs_license_num`: this is the Taxi & License Commission license number of the company operating the trip. Possible values are HV0002 (Juno), HV0003 (Uber), HV0004 (Via), HV0005 (Lyft).
- `request_datetime`, `on_scene_datetime`, `pickup_datetime`, `dropoff_datetime`: logs datetime for ride request, when driver arrived, picked-up & dropped off passenger(s).
- `PULocationID`, `DOLocationID`: where the trip began & ended. Those are `int` values.
- `trip_miles`, `trip_time`: miles for passenger trip & total time in seconds of trip
- `base_passenger_fare`: base fare excluding toll (`tolls`), tips (`tips`), taxes (`sales_tax`) and fees (`airport_fee`, `congestion_surcharge`, `bcf`). 
- `driver_pay`: total driver pay (exclusing tools, tips, commission, taxes...)
- `shared_match_flag`: did the passenger share the vehicle with another passenger who booked separately? (Y/N)

Let's load it and print the first rows.

In [ ]:
import pandas as pd

data = pd.read_parquet("data/taxi-trips-2022-01.parquet")
data.head()

### 1-3. Exploring our data

Let's assume our goal will be to create an application able to predict the fare of a trip, from the pick-up & dropoff locations. We will eventually be using the following columns:
- `base_passenger_fare`: our target variable
- `hvfhs_license_num`: fare might depend on operating company (HV0003 or HV0005)
- `request_datetime`, `on_scene_datetime`, `pickup_datetime`, `dropoff_datetime`: fare might depend on congestion & time of pickup
- `PULocationID`, `DOLocationID`: fare will depend on pick up and drop off location
- `trip_miles`, `trip_time`: these fields might be useful to normalize training data

Let's first explore quality of these key fields. What can you see? Is data quality sufficient?

**Conclusion:** : 
- TODO

## 2 - Installing Great Expectations

Great expectation allows us to:
- define data quality rules in a language agnostic format (as config files)
- run these data quality checks & rules on various types of data sources
- trigger actions & alerting whenever a rule breaks
- generate data quality reports easily from our set of rules

Your environment should already contain great expectation as a python library. Otherwise you can simply follow the following commands to install it: https://docs.greatexpectations.io/docs/guides/setup/installation/local

In [ ]:
%pip list | grep great-expectations

## 3 - Getting to know Great Expectations

### 3-1. Connecting to our data

As we will see, Great expectations, works with a lot of configuration files (`.yml`, `.json`). This enables us to stay language & datasource agnostic, and to have our rules & checks documented as config and not hard coded.

The main entrypoint & best practice to manage 'rules' is to have a folder `gx` where we will store all our config. 

Before starting implementing checks & triggers, we first need to connect to a dataset, and explain to Great Expectation how to connect to it. This can usually be best done in the following main file: `gx/great_expectations.yml`. 

In [ ]:
import yaml
from pprint import pprint

with open("great_expectations/great_expectations.yml", "r") as stream:
    try:
        ge_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

pprint(ge_config["datasources"], indent=0)

We have already made part of our task: and told great expectation where to find our dataset, and how to read it (using Pandas & the parquet read function).

### 3-2. Writing a first expectation
Data quality rules (or "expectations") can also be written in config files and are stored in the `gx/expectations/` folder.
We have already written one expecting the base fare not to be negative.

In [ ]:
import json

# Open JSON file
with open("great_expectations/expectations/taxi-trips-expectations.json", "r") as f:
    data = json.load(f)

# Pretty print JSON data
print(json.dumps(data, indent=4))

You will have to define your own expectations afterwards, feel free to [explore the doc](https://docs.greatexpectations.io/docs/guides/expectations/how_to_create_and_edit_expectations_based_on_domain_knowledge_without_inspecting_data_directly) to understand the JSON definition of expectations.

### 3-3. Checking our data
Now that we can connect to our data... and have defined a set of data quality rules, how do we apply these rules to our datasources? As you would expect, great expectations also uses configuration files to run data checks, as found in the `great_expectations/checkpoints/` folder. Where we bin a datasource (and particularly a data asset) to a suite of expectations.

In [ ]:
with open("great_expectations/checkpoints/taxi-trips-checkpoint.yml", "r") as stream:
    try:
        chkp_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

pprint(chkp_config)

Again, take some time to [follow the documentation](https://docs.greatexpectations.io/docs/guides/validation/checkpoints/how_to_create_a_new_checkpoint/) to understand the content of this file.

Before running our checkpoint, let's introduce the `great_expectations.data_context`: this object scans your repository and stores all datasources, checkpoints & expectations you have defined. You can then handle them from your code.

In [ ]:
import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.datasource.types import BatchKwargs

import datetime

# context = gx.get_context()
context = gx.data_context.DataContext()
print(context.list_expectation_suite_names())
print([datasource["name"] for datasource in context.list_datasources()])
print(context.list_checkpoints())

In [ ]:
context.list_checkpoints()

You can now run a checkpoint

In [ ]:
context.run_checkpoint(checkpoint_name="taxi-trips-checkpoint")

Note that GE allows you to export your results in a simple html format

In [ ]:
context.open_data_docs()

## 4 - More expectations & more data!

### 4-1. More expectations
Now use what you have learnt to great 2 or 3 more expectations for your data. You can look for ideas there: https://greatexpectations.io/expectations/

TODO : Create your expectations in the `great_expectations/expectations/taxi-trips-expectations.json` file and once it's done run the code below to make sure they works.


In [ ]:
import json

with open("great_expectations/expectations/taxi-trips-expectations.json") as f:
    expectation = json.load(f)

pprint(expectation["expectations"])

### 4-2. Running our new expectations
Update your checkpoint file & run the expectations you have just created. 

In [ ]:
context.run_checkpoint(checkpoint_name="taxi-trips-checkpoint")
context.open_data_docs()

## 5 - Wrapping up

In this short tutorial, you have seen how to configure a simple great expectations project & run a few data quality rules. The main takeaway is that GE allows you to create expectations & run them entirely with configuration, abstracting the connection to data sources behind.

Other exercices you could work on:
- Connecting to a distant datasource (s3, BigQuery...)
- Writing your own expectation (not available in the gallery)
- Using great expectations actions to avoid deploying if data quality is not as expected
- Automate and schedule Data Quality test with Prefact (or any other orchestrator tool)